In [3]:
#coding=utf-8
from keras.models import Model
from keras.layers import Input,Dense,Dropout,BatchNormalization,Conv2D,MaxPooling2D,AveragePooling2D,concatenate
from keras.layers.convolutional import Conv2D,MaxPooling2D,AveragePooling2D
import numpy as np
seed = 7
np.random.seed(seed) 

In [8]:
num_BN = 1
def Conv2d_BN(x, nb_filter,kernel_size, padding='same',strides=(1,1),name=None):
    if name is None:
        name = str(num_BN)
        num_BN += 1  
        
    x = Conv2D(nb_filter,kernel_size,padding=padding,strides=strides,activation='relu',name=name + "_Conv")(x)
    x = BatchNormalization(axis=3,name=name + "_NB")(x)
    return x

In [9]:
def Inception(x, nb_filter, Inception_no=None):  
    
    branch1x1 = Conv2d_BN(x,nb_filter,kernel_size=(1,1), padding='same',strides=(1,1), name="Inception_%d_branch1x1_1"%(Inception_no))
 
    branch3x3 = Conv2d_BN(x,nb_filter,kernel_size=(1,1), padding='same',strides=(1,1),name="Inception_%d_branch1x1_2"%(Inception_no))
    branch3x3 = Conv2d_BN(branch3x3,nb_filter,kernel_size=(3,3), padding='same',strides=(1,1),name="Inception_%d_branch3x3_1"%(Inception_no))
 
    branch5x5 = Conv2d_BN(x,nb_filter,kernel_size=(3,3), padding='same',strides=(1,1),name="Inception_%d_branch3x3_2"%(Inception_no))
    branch5x5 = Conv2d_BN(branch5x5,nb_filter,kernel_size=(3,3), padding='same',strides=(1,1),name="Inception_%d_branch3x3_3"%(Inception_no))
 
    Inception_name = "Inception_no%d"%(Inception_no) 
    branchpool = MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same', name=Inception_name+"_pool")(x)
    branchpool = Conv2d_BN(branchpool,nb_filter,kernel_size=(1,1),padding='same',strides=(1,1),name="Inception_%d_branch1x1_3"%(Inception_no))
     
    x = concatenate([branch1x1, branch3x3, branch5x5, branchpool],axis=3, name=Inception_name+"_Concat")
    return x 

In [10]:
 inpt = Input(shape=(224,224,3), name="Input")
#padding = 'same'，填充为(步长-1）/2,还可以用ZeroPadding2D((3,3))
x = Conv2d_BN(inpt,64,(7,7),strides=(2,2),padding='same', name="NB1")
 
x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same', name="pool1")(x)
x = Conv2d_BN(x,192,(3,3),strides=(1,1),padding='same', name="NB2")
 
x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same', name="pool2")(x)
x = Inception(x,64, Inception_no=1)#256
x = Inception(x,120, Inception_no=2)#480
 
x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same', name="pool3")(x)
x = Inception(x,128, Inception_no=3)#512
x = Inception(x,128, Inception_no=4)
x = Inception(x,128, Inception_no=5)
x = Inception(x,132, Inception_no=6)#528
x = Inception(x,208, Inception_no=7)#832
 
x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same', name="pool4")(x)
x = Inception(x,208, Inception_no=8)
x = Inception(x,256, Inception_no=9)#1024
 
x = AveragePooling2D(pool_size=(7,7),strides=(7,7),padding='same')(x)
x = Dropout(0.4, name="Dropout1")(x)
x = Dense(1000,activation='relu', name="Dense1")(x)
x = Dense(1000,activation='softmax', name="Dense2")(x)
model = Model(inpt,x,name='inception')
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
model.summary()

# 把模型保存为图片
from keras.utils import plot_model
plot_model(model,to_file='model_png/205GoogleNet.png', show_layer_names=True, show_shapes=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
NB1_Conv (Conv2D)               (None, 112, 112, 64) 9472        Input[0][0]                      
__________________________________________________________________________________________________
NB1_NB (BatchNormalization)     (None, 112, 112, 64) 256         NB1_Conv[0][0]                   
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 56, 56, 64)   0           NB1_NB[0][0]                     
__________________________________________________________________________________________________
NB2_Conv (